In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
import seaborn as sns

In [ ]:
dataset=pd.read_csv('/content/framingham.csv')
dataset.drop(['education'],axis=1,inplace=True)
dataset.head()


In [ ]:
dataset.dropna(axis=0, inplace=True)
dataset.shape
sns.countplot(x='TenYearCHD',data=dataset)
plt.show()
cases = dataset.TenYearCHD.value_counts()
print(f"There are {cases[0]} patients without heart disease and {cases[1]} patients with the disease")

In [ ]:
plt.figure(figsize=(15,8))
sns.heatmap(dataset.corr(), annot = True)
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
!pip install boruta
from boruta import BorutaPy
X = dataset.iloc[:,:-1].values
y = dataset.iloc[:,-1].values

forest = RandomForestClassifier(n_estimators=1000, n_jobs=-1, class_weight='balanced')

# define Boruta feature selection method
feat_selector = BorutaPy(forest, n_estimators='auto', verbose=2)

# find all relevant features
feat_selector.fit(X, y)

In [ ]:
# show the most important features
most_important = dataset.columns[:-1][feat_selector.support_].tolist()
most_important
top_features = dataset.columns[:-1][feat_selector.ranking_ <=6].tolist()
top_features

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from collections import Counter
X = dataset[top_features]
y = dataset.iloc[:,-1]
# the numbers before smote
num_before = dict(Counter(y))

#perform smoting

# define pipeline
over = SMOTE(sampling_strategy=0.8)
under = RandomUnderSampler(sampling_strategy=0.8)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)

# transform the dataset
X_smote, y_smote = pipeline.fit_resample(X, y)


#the numbers after smote
num_after =dict(Counter(y_smote))
print(num_before, num_after)

In [ ]:
new_data = pd.concat([pd.DataFrame(X_smote), pd.DataFrame(y_smote)], axis=1)
new_data.columns = ['age', 'totChol', 'sysBP', 'diaBP', 'BMI', 'heartRate', 'glucose','TenYearCHD']
new_data.head()
X_new = new_data[top_features]
y_new= new_data.iloc[:,-1]
X_new.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_new,y_new,test_size=.2,random_state=42)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_train = pd.DataFrame(X_train_scaled)

X_test_scaled = scaler.transform(X_test)
X_test = pd.DataFrame(X_test_scaled)

In [ ]:

dt_scores=[]
cr_scores=[]
for cr in ['gini', 'entropy']:
  for i in range(1,len (X.columns)+1):
    dt_classifier=DecisionTreeClassifier (criterion=cr,max_features=i, random_state=42)
    dt_classifier.fit(X_train,y_train)
    dt_scores.append(dt_classifier.score (X_test, y_test))
  print (f'BEST MAX_Features for {cr}: {np.argmax (dt_scores)+1}')
  cr_scores.append(dt_scores [np.argmax(dt_scores)])
print (f'Best Criterion: {"gini" if not np.argmax(cr_scores) else "entropy"}')

In [ ]:
dt_classifier=DecisionTreeClassifier (criterion='gini', max_features=12, random_state=42)
dt_classifier.fit(X_train,y_train)
b=dt_classifier.score(X_test,y_test)
print(b)

In [ ]:
svc_scores=[]
kernels=['linear', 'poly', 'rbf', 'sigmoid']
for i in range(len(kernels)):
  svc_scores_c=[]
  for ch in range(1,11):
    if kernels[i]=='poly':
      svc_scores_poly=[]
      for d in range(3,10):
        svc_classifier=SVC (kernel=kernels[i],C=ch, degree=d)
        svc_classifier.fit(X_train, y_train)
        svc_scores_poly.append(svc_classifier.score (X_test,y_test))
      print (f'Best polynomial score: {np.argmax (svc_scores_poly)+3}')
      svc_scores_c.append(svc_scores_poly [np.argmax (svc_scores_poly)])
    else:
      svc_classifier=SVC (kernel=kernels[i], C=ch)
      svc_classifier.fit(X_train, y_train)
      svc_scores_c.append(svc_classifier.score (X_test, y_test))
  print (f'Best choice of c for {kernels[i]}: {np.argmax (svc_scores_c)+1}')
  svc_scores.append(svc_scores_c[np.argmax(svc_scores_c)])
print(f'Best choice of k: {kernels [np.argmax(svc_scores)]}')

In [ ]:
svc_classifier= SVC (kernel='rbf', C=9)
svc_classifier.fit(X_train,y_train)
a=svc_classifier.score (X_test,y_test)
print(a)

In [ ]:
#RandomForest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
rf_clf = RandomForestClassifier(n_estimators=1000, random_state=42)
rf_clf.fit(X_train, y_train)
rf_pred=rf_clf.predict(X_test)
rf_accuracy=accuracy_score (y_test,y_pred)
print (rf_accuracy*100)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

In [ ]:

#NaiveBayes
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
classifier= GaussianNB()
classifier.fit(X_train, y_train)
naive_pred = classifier.predict(X_test)
naive_accuracy=metrics.accuracy_score (y_test,naive_pred)
print('Accuracy Score:')
print(naive_accuracy*100)

from sklearn.metrics import classification_report
print(classification_report(y_test,naive_pred))

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Example: Replace X, y with your dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train XGBoost
xgb_model = XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)

# Predictions
y_pred = xgb_model.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
import matplotlib.pyplot as plt

# Example data: Replace with your actual results
models = ['Naive Bayes', 'SVM', 'Decsion Trees', 'XGBoost','Random Forest']  # Names of your algorithms
accuracies = [0.79, 0.823, 0.825,0.83,0.827]  # Replace with accuracy values for each model

# Plotting the accuracies
plt.figure(figsize=(10, 6))
plt.bar(models, accuracies, color='skyblue')
plt.xlabel('Algorithms', fontsize=14)
plt.ylabel('Accuracy', fontsize=14)
plt.title('Comparison of Algorithm Accuracies', fontsize=16)
plt.ylim(0, 1)  # Set limit based on your accuracy scale (0-1 or 0-100)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()